In [1]:
import bson
import pandas as pd
import datetime
from bson.objectid import ObjectId
from itertools import groupby
from operator import itemgetter


In [2]:
with open('test/datarows.bson', 'rb') as f:
    datarows = bson.decode_all(f.read())


with open('test/devices.bson', 'rb') as f:
    devices = bson.decode_all(f.read())

import bson

with open('test/animals.bson', 'rb') as f:
    animals = bson.decode_all(f.read())

In [3]:
# Crear un diccionario donde la clave es el "deviceMACAddress" y el valor es el diccionario completo del dispositivo correspondiente
device_dict = {}
for device in devices:
    device_dict[device["deviceMACAddress"]] = device

# Crear una nueva lista de datos fusionando los datos de datarows y devices
merged_data = []
for row in datarows:
    if row["UUID"] in device_dict:
        device_data = device_dict[row["UUID"]]
        merged_row = {**row, **device_data} # Fusionar los dos diccionarios en uno solo
        merged_data.append(merged_row)

In [4]:
#MERGE CON ANIMALS

for row in merged_data:
    animal_id = row.get('deviceAnimalID')
    animal = next((a for a in animals if a.get('_id') == animal_id), None)
    if animal:
        row['animal'] = animal
    else:
        row['animal'] = None

In [6]:
#TO DATE TIME

for d in merged_data:
    timestamp_str = d['dataRowData']['timestamp']
    try:
        timestamp_obj = pd.to_datetime(timestamp_str)
    except ValueError:
        try:
            timestamp_obj = pd.to_datetime(timestamp_str, format='%Y-%d-%mT%H:%M:%S')
        except ValueError:
            timestamp_obj = timestamp_obj
    d['dataRowData']['timestamp'] = timestamp_obj

In [8]:
def gps(desired_id: str, date_str: str):
    # Convertir la fecha en un objeto datetime.date
    date_obj = datetime.datetime.strptime(date_str, "%Y, %m, %d").date()
    
    # Convertir el string de desired_id en un objeto ObjectId
    desired_id_obj = ObjectId(desired_id)
    
    #FILTRO POR ID CAMPO y DATAROWTYPE = GPS
    gps_data = [element for element in merged_data if element.get("dataRowType") == "GPS" and desired_id_obj in element["animal"]["animalSettlement"]]
    #FILTRO POR DIA
    filtered_data = [d for d in gps_data if d["dataRowData"]["timestamp"] == date_obj]


    # Contar UUIDs/COLLARES únicos ese día
    unique_uuids = set(d['UUID'] for d in filtered_data)
    

    # CREO DICCIONARIO KEYS= COLLAR, INFO LAT LONG DATE
    gps_data_by_uuid = {}
    for uuid in unique_uuids:
        filtered_gps_data_animal = [d for d in filtered_data if d['UUID'] == uuid]
        gps_data_dict = {'timestamp': [], 'latitud': [], 'longitud': []}
        for d in filtered_gps_data_animal:
            gps_data_dict['timestamp'].append(d['dataRowData']['timestamp'])
            gps_data_dict['latitud'].append(d['dataRowData']['lat'])
            gps_data_dict['longitud'].append(d['dataRowData']['lng'])
        gps_data_by_uuid[uuid] = gps_data_dict
 

    
    return (gps_data_by_uuid)


In [9]:
collardicc = gps("620e6e5e60543d0026a01f0e","2022, 9, 21")

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_9136\2991580004.py:11: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  filtered_data = [d for d in gps_data if d["dataRowData"]["timestamp"] == date_obj]


In [10]:
collardicc.keys()

dict_keys(['0004A30B00F82BD2', '0004A30B00F82E9C', '0004A30B00F88969', '0004A30B00F825B7', '0004A30B00EE959D'])

In [11]:
collardicc["0004A30B00F825B7"].keys()

dict_keys(['timestamp', 'latitud', 'longitud'])

In [ ]:
print(collardicc["0004A30B00F825B7"]["timestamp"][0])

print(collardicc["0004A30B00F825B7"]["latitud"][0])

print(collardicc["0004A30B00F825B7"]["longitud"][0])

2022-09-21 00:53:02
-32.85475433333333
-63.6427645


In [ ]:
def caravanas(desired_id: str, date_str: str):
    # Convertir la fecha en un objeto datetime.date
    date_obj = datetime.datetime.strptime(date_str, "%Y, %m, %d").date()

    # Convertir el string de desired_id en un objeto ObjectId
    desired_id_obj = ObjectId(desired_id)

    # Filtrar por ID CAMPO y DATAROWTYPE = CARAVANA
    beacon_data = [element for element in merged_data if element.get("dataRowType") == "BEACON" and desired_id_obj in element["animal"]["animalSettlement"]]

    # Filtrar por día
    filtered_dataB = [d for d in beacon_data if d["dataRowData"]["timestamp"].date() == date_obj]

    
    # Contar UUIDs/COLLARES únicos ese día
    unique_uuids = set(d['UUID'] for d in filtered_dataB)

    collar_data = {}
    collar_list = set(d['UUID'] for d in filtered_dataB)

    for collar in collar_list:
        #FILTRO POR COLLAR
        beacon_data1 = [element for element in filtered_dataB if element.get("UUID") == collar]
        
        data_list = []
        
        for d in beacon_data1:
            date = d['dataRowData']['timestamp']
            mac = d['dataRowData']['mac']
            data_list.append({'date': date, 'mac': mac})
        
        
        # Ordenar la lista por fecha
        data_list_sorted = sorted(data_list, key=itemgetter('date'))
        
        # Agrupar por hora
        grouped_data = []
        for hour, group in groupby(data_list_sorted, lambda x: x['date'].strftime('%Y-%m-%d %H')):
            macs = set()
            for item in group:
                macs.add(item['mac'])
            grouped_data.append({'hour': hour, 'macs': macs})
        
        collar_data[collar] = grouped_data

    return collar_data

In [ ]:
collardicc1 = caravanas("620e6e5e60543d0026a01f0e","2022, 9, 21")

In [ ]:
collardicc1.keys()

dict_keys(['0004A30B00F825B7', '0004A30B00F82BD2', '0004A30B00F82E9C', '0004A30B00EE959D'])

In [ ]:
collardicc1["0004A30B00F825B7"][0]

{'hour': '2022-09-21 12', 'macs': {'A7AA2D0FF0F3'}}

In [ ]:
collardicc1["0004A30B00F825B7"][1]

{'hour': '2022-09-21 18', 'macs': {'7F93E1ED74CA'}}

In [ ]:
collardicc1["0004A30B00F82E9C"][0]["macs"]

{'1D96FE9FFEF4', '3B415B321DDA', 'F356641DB6D3'}